# Simulated Observation of a Galactic Center Deep Exposure with AXIS

## Introduction

### Objective

Using the `SIXTE` software package, we will simulate an AXIS pointing that reproduces a _Chandra_ observation of the Galactic Center (GC):

ObsID = 3392

Exp. = 167 ks

R.A. = 266.417 = 17:45:40

Dec. = -29.008 = -29:00:38 

Roll = 75.30569

### Required Software

It is assumed that the software listed below are installed and initialized in a terminal:

* `HEAsoft` (6.34)

    https://heasarc.gsfc.nasa.gov/docs/software/lheasoft


* `SIXTE` and `SIMPUT` (3.0.5 and 2.6.3, respectively)

    https://www.sternwarte.uni-erlangen.de/sixte/installation-steps


* AXIS-specific `SIXTE` files (3.1.0):

    https://www.sternwarte.uni-erlangen.de/sixte/instruments

* `CIAO` (4.17)

    https://cxc.cfa.harvard.edu/ciao

    (which will be used to create a comparison image of real _Chandra_ data, and to exclude point sources from the publicly-available diffuse image.)

### General pipeline of a `SIXTE` simulation:


1. CREATE XSPEC MODELS

    Run `Xspec` to make a spectral model for the diffuse emission, as well as a typical model for all point sources, or one for each source.

2. CREATE SIMPUT FILES

    Run `simputfile` to generate a `SIMPUT` file for each source based on `Xspec` files from Step 1.

3. MERGE SIMPUT FILES

    Run `simputmerge` to merge the `SIMPUT` files of all sources from Step 2.

4. SIMULATE THE EVENTS FILE

    Run `sixtesim` to generate a simulated events file from the merged `SIMPUT` file from Step 3.

5. GENERATE THE IMAGE FILE

    Run `imgev` to generate an image file from the simulated events file from Step 4.

6. GENERATE THE EXPOSURE MAP

    Run `exposure_map` to generate an exposure file.

## 1. CREATE XSPEC MODELS

### 1.1. POINT SOURCES

The point source catalog is that of [Muno et al. (2009)](https://ui.adsabs.harvard.edu/abs/2009ApJS..181..110M/abstract):

https://ui.adsabs.harvard.edu/abs/2009ApJS..181..110M/abstract

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJS/181/110

In the [Vizier page](https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJS/181/110) of the [Muno et al. (2009)](https://ui.adsabs.harvard.edu/abs/2009ApJS..181..110M/abstract) catalog, restrict the search to a circle of $r = 17^{\prime}$ centered at the position given in the Objective. This is the minimum radius that fully encompasses all sources in the $24^{\prime}$ AXIS FoV regardless of the selected roll angle. Even though this includes a number of superfluous sources that land outside the AXIS FoV, they will be ignored by the `SIXTE` simulation software. 

Select the columns `CXOGC`, `RAJ2000`, `DEJ2000`, `Sflux`, and `Hflux`, then save the data in "ASCII text/plain" format as `src_list.txt`.

There should be 6,469 sources. Erase most of the irrelevant header rows and comment the column headers/units so that `src_list.txt` looks like this:

<div>
<img src="img/src_list.png" width="800"/>
</div>


Some of the flux values are negative because these are background-subtracted fluxes from a model fit to the data, which may have no counts detected above or below 2 keV due to the source type (soft vs. hard emitters) and line-of-sight absorption.

We want the simulation to assume a broadband flux (0.5-8 keV). However, directly summing the soft and hard-band fluxes leads to a problem: a handful of sources (e.g., CXOGC 174513.6-290214) end up with a broadband flux of 0.

One option is to add the soft and hard flux values in quadrature.

Instead, we will simply assume that negative flux values are 0, i.e., equivalent to the background, before summing.

Consider the following comments by [Muno et al. (2009)](https://ui.adsabs.harvard.edu/abs/2009ApJS..181..110M/abstract):

Section 2.4:
    
>_"We are sensitive to 4e32 erg/s (=5.2e-14 erg/cm2/s in 0.5-8 keV, assuming D=8kpc) at 90% conf over 1 deg2, and 1e32 (=1.3e-14 erg/cm2/s) over 0.1 deg2."_

Section 3.1: 

>_"We have assumed a distance of 8 kpc and 6E22 /cm2 of absorption."_

>_"Photon fluxes can be converted to energy fluxes according to 1 ph/cm2/s = 8.7E-9 erg/cm2/s (0.5-8 keV)."_

Therefore, we create a typical `Xspec` model that will be used for all point sources.

This will be an absorbed power law whose parameters are:

* $N_{\rm H} = 6 \times 10^{22}$; equal to the expected line-of-sight value.

* $\Gamma = 1.5$; typical value.

* Norm. = $10^{-5}$; adjusted for each source according to its cataloged flux value.

Edit and paste the following `Xspec` commands to a file called `sed_point_cmd.xcm`:

In [ ]:
abund wilm
xsect vern
query y
set par {6 & 1.5 & 1E-5}
model tbabs*pow & $par
save mo sed_point.xcm
exit

In a `HEAsoft`-initialized terminal, call `Xspec` to create the model file for the point emission from the command file `sed_point_cmd.xcm`:

In [ ]:
xspec - sed_point_cmd.xcm

This creates an `Xspec` model file for the point sources (`sed_point.xcm`).

#### ALTERNATIVE: Using an Existing SIMPUT File

An alternative to using a single model for all point sources is to use different models based on the source hardness ratios. 

In fact, there exists a ready-made `SIMPUT` file of the GC (`bremms.simput`) that offers this enhanced feature:

https://www.sternwarte.uni-erlangen.de/sixte/example-source-files/

This `SIMPUT` file was merged with that of the diffuse emission (`diffuse.simput`) in order to create the simulated AXIS RGB image shown in the Appendix.

### 1.2. DIFFUSE EMISSION

Unless the diffuse emission is uniform, then an image or model describing the spatial distribution of this emission is required. 

The public _Chandra_ image of Sagittarius A* by Baganoff et al. (2009) will serve this purpose:

https://chandra.harvard.edu/photo/openFITS/xray_data.html

<div>
<img src="img/chandra_sgra.jpg" width="800"/>
</div>

There are 3 files each representing a different non-overlapping energy band (equivalent to RGB for soft X-rays):

* `sgra_200-3300_eV.fits`

* `sgra_3300-4700_eV.fits`

* `sgra_4700-8000_eV.fits`

They need to be merged into a single-band image (`sgra_200-8000_eV.fits`) using the `HEAsoft` tool `fimgmerge`.

Edit and paste the following commands into a `HEAsoft`-initialized terminal:

In [ ]:
export file_1='sgra_200-3300_eV.fits'
export file_2='sgra_3300-4700_eV.fits'
export file_3='sgra_4700-8000_eV.fits'
export file_out='sgra_200-8000_eV.fits'

fimgmerge \
    infile=${file_1} \
    list=${file_2},${file_3} \
    outfile=${file_out} \
    xoffset=0,0 \
    yoffset=0,0 \
    clobber=y 

There should now be a broad-band _Chandra_ image (`sgra_200-8000_eV.fits`).

In addition to the diffuse emission that interests us, this _Chandra_ image contains bright point sources whose presence will affect our model of the diffuse flux distribution. 

To obtain a suitable image for the diffuse emission, we need to remove the point sources. Sources can be removed with the `CIAO` tool `dmfilth` which requires a stacked list of regions representing the point sources in physical detector coordinates specific to _Chandra_. 

So the point source catalog from the previous subsection (`src_list.txt`) will have to be transformed into a `DS9` region file with physical detector coordinates. 

First, we create a standard region file (`src_list_RADEC.reg`) based on equatorial coordinates that define circular regions of 2 $^{\prime\prime}$ radius.

Edit and run the script `make_region.sh` to create a region file for the point sources:

In [ ]:
#!/bin/bash

# Input file and starting row
src_list="src_list.txt"
start_row="15"

# Output region file
output_file="src_list_RADEC.reg"

# Write the header to the output file
echo "# Region file format: DS9 version 4.1" > "${output_file}"
echo "global color=green width=1" >> "${output_file}"
echo "fk5" >> "${output_file}"

# Read each row of the source list beginning from start_row
tail -n +${start_row} "${src_list}" | while IFS=' ' read -r id ra dec soft hard
do
    # Append each row to the output file
    echo "circle(${ra},${dec},2\")" >> "${output_file}"
done


There should now be a region file (`src_list_RADEC.reg`) that looks like:

<div>
<img src="img/src_list_RADEC.png" width="300"/>
</div>

Open the diffuse image (`sgra_200-8000_eV.fits`) in `DS9` and load the region file (`src_list_RADEC.reg`):

In [ ]:
ds9 sgra_200-8000_eV.fits -region src_list_RADEC.reg &

In `DS9`, save the region file in physical coordinates as `src_list_XY.reg`.

Open `src_list_XY.reg` with an editor and delete the header rows so that it looks like:

<div>
<img src="img/src_list_XY.png" width="300"/>
</div>

In a `CIAO`-initialized terminal, run the `CIAO` tool `DMFILTH` with the `BILINT` method (i.e., bilateral interpolation) which takes an input catalogue of point sources (`src_list_XY.reg`), removes them from the image (`sgra_200-8000_eV.fits`), and outputs a point-source-excluded diffuse image (`sgra_200-8000_eV_diffuse.fits`):

In [ ]:
dmfilth \
 infile=sgra_200-8000_eV.fits \
 outfile=sgra_200-8000_eV_diffuse.fits \
 method=BILINT \
 srclist=@src_list_XY.reg \
 bkglist=""

There should now be a diffuse image called `sgra_200-8000_eV_diffuse.fits`.

Using the `HEAsoft` tools `fdump` and `ftlist`, check the header and contents to ensure that any keywords from the original file (`sgra_200-8000_eV.fits`) are present in the diffuse image (`sgra_200-8000_eV_diffuse.fits`):

In [ ]:
fdump sgra_200-8000_eV.fits head.txt columns="" rows=- clobber=y

ftlist infile=sgra_200-8000_eV_diffuse.fits option=I outfile=data.lis colheader=no rownum=no

The image below compares the original full-band _Chandra_ image (left panel: `sgra_200-8000_eV.fits`) with the diffuse image (right panel: `sgra_200-8000_eV_diffuse.fits`) and shows that the removal of point sources was successful:

<div>
<img src="img/chandra_sgra_original_vs_source_removed.png"/>
</div>

Now that we have an image of the diffuse emision, we can create its `Xspec` model. 

Table 3 of [Muno et al. (2004)](https://ui.adsabs.harvard.edu/abs/2004ApJ...613..326M/abstract) lists parameters for the best-fitting model of the diffuse emission, which we also adopt here, i.e., an absorbed, two-temperature plasma (0.5 and 8 keV) with two Fe lines (6.4 and 6.7 keV).

Edit and paste the following `Xspec` commands to a file called `sed_diffuse_cmd.xcm`:

In [ ]:
abund wilm
xsect vern
query y
set par {6 & 1 & 1.4E-3 & 8 & 1.4E-3 & 6.4 & 0.04 & 4E-3 & 6.7 & 0.04 & 4E-3}
model tbabs*(bbody+bbody+gauss+gauss) & $par
flux 2 8
save mo sed_diffuse.xcm
exit

In a `HEAsoft`-initialized terminal, call `Xspec` to create the model file for the diffuse emission from the command file `sed_diffuse_cmd.xcm`:

In [ ]:
xspec - sed_diffuse_cmd.xcm

This creates an `Xspec` model file for the diffuse emission (`sed_diffuse.xcm`).

The output of the `Xspec` command `flux 2 8` is:

In [ ]:
 Model Flux  0.017389 photons (1.4448e-10 ergs/cm^2/s) range (2.0000 - 8.0000 keV)

According to Table 3 of [Muno et al. (2004)](https://ui.adsabs.harvard.edu/abs/2004ApJ...613..326M/abstract), the average of the summed (soft+hard) observed fluxes in 2-8 keV is $2.4 \times 10^{-13}$ erg cm $^{-2}$ s $^{-1}$ arcmin $^{-2}$. 

For the $24^{\prime}$-wide AXIS FoV, this converts to a flux of $1.4 \times 10^{-10}$ erg cm $^{-2}$ s $^{-1}$.

Therefore, the normalization parameters of the different model components were adjusted until the `Xspec` output flux in 2-8 keV equaled this value.

## 2. CREATE SIMPUT FILES

It is assumed that the `Xspec` model files for point sources and for the diffuse emission were made according to the instructions in Section 1, and that any parameters for these sources are correctly set in `config.py` if using the Python scripts.

We need to call the `SIXTE` command `simputfile` in order to create a `SIMPUT` file for each point source, as well as one for the diffuse emission.

In a `SIXTE`-initialized terminal, edit and run the script `make_simput_diffuse.sh` to create a `SIMPUT` file for the diffuse source:

In [ ]:
#!/bin/bash

# Input source list and parameters
file_name='diffuse.simput'
src_name='diffuse'
ra='266.417'
dec='-29.008'
flux='1.4e-10'
elow='0.1'
eup='15'
nbins='1490'
emin='0.5'
emax='8'
xspec='sed_diffuse.xcm'
img='sgra_200-8000_eV_diffuse.fits'

# Run the command
simputfile \
    Simput=${file_name} \
    Src_Name=${src_name} \
    RA=${ra} \
    Dec=${dec} \
    srcFlux=${flux} \
    Elow=${elow} \
    Eup=${eup} \
    NBins=${nbins} \
    Emin=${emin} \
    Emax=${emax} \
    XSPECFile=${xspec} \
    ImageFile=${img} \
    clobber=yes

And edit and run the script `make_simput_point.sh` to create `SIMPUT` files for the point sources in `src_list.txt`:

In [ ]:
#!/bin/bash

# Input source list and parameters
src_list="src_list.txt"
start_row="15"
xspec="sed_point.xcm"
elow="0.1"
eup="15"
nbins="1490"
emin="0.5"
emax="8"
fluxconv="8.7e-9"

# Read file starting from row number "start_row"
tail -n +${start_row} "${src_list}" | while IFS=' ' read -r id ra dec soft hard
do
    
    # Apply clipping: set soft and hard fluxes to 0 if they are less than 0
    if (( $(echo "$soft < 0" | bc -l) )); then
        soft=0
    fi

    if (( $(echo "$hard < 0" | bc -l) )); then
        hard=0
    fi
    
    # Calculate total photon flux (flux) as the sum of soft and hard bands
    flux=$(echo "$soft + $hard" | bc -l)
    
    # Multiply photon flux by the flux conversion factor to get cgs units
    converted_flux=$(echo "$flux * $fluxconv" | bc -l)
    
    # Format output for scientific notation with 3 decimal places
    formatted_flux=$(printf "%1.3e" $converted_flux)

    # Run the command
    simputfile \
        Simput=${id}.simput \
        Src_Name=${id} \
        RA=${ra} \
        Dec=${dec} \
        srcFlux=${formatted_flux} \
        Elow=${elow} \
        Eup=${eup} \
        NBins=${nbins} \
        Emin=${emin} \
        Emax=${emax} \
        XSPECFile=${xspec} \
        clobber=yes
    
done


There should now be a `SIMPUT` file for the diffuse emission (`diffuse.simput`), and one for each point source (e.g., `174457.4-285622.simput`). 

## 3. MERGE SIMPUT FILES

The `SIMPUT` files made in Section 3 for the diffuse source (`diffuse.simput`) and for the point sources (e.g., `174457.4-285622.simput`) need to be merged into a single file (`GC.simput`) with the `SIXTE` tool `simputmerge`.

Edit and run the script `merge_simput.sh` in a `SIXTE`-initialized terminal:

In [ ]:
#!/bin/bash

# Input list of point sources and starting row
src_list="src_list.txt"
start_row="15"

# Input SIMPUT file of diffuse source
diffuse_file='diffuse.simput'

# Output SIMPUT file
output_file="GC.simput"

# Initialize an empty csv_string starting with the diffuse source
csv_string="${diffuse_file}"

# Read the source list starting from the specified row
tail -n +${start_row} "${src_list}" | while IFS=' ' read -r id ra dec soft hard
do
    # Append the current ID with ".simput" to the csv_string
    csv_string="${csv_string},${id}.simput"
done

# Run the command
simputmerge \
    Infiles=${csv_string} \
    Outfile=${output_file} \
    FetchExtensions=yes \
    clobber=yes

There should now be a single `SIMPUT` file  (`GC.simput`) that contains the diffuse source and all the point sources:


<div>
<img src="img/GC_simput.png" width="1000"/>
</div>

This is the required input file for `SIXTE` to simulate an events file.

## 4. SIMULATE THE EVENTS FILE

### 4.1. ATTITUDE FILE

The distance between adjacent detector chips of AXIS is about 40 $^{\prime\prime}$. Therefore, a long-exposure image leaves a thick cross pattern through the center of the FoV.

To overcome this problem, it is recommended that the telescope perform a dithering pattern. This smears out the gaps between the detector chips, which creates a more esthetically-pleasing image. An added benefit is that it helps reduce systematic errors.

The `SIXTE` tool `attgen_dither` will create an attitude file given a pointing direction, a dithering frequency, and an amplitude.

Edit and run the script `make_attitude.sh` in a `SIXTE`-initialized terminal:

In [ ]:
#!/bin/bash

# Input parameters
att_file='GC_AXIS_100000_att.fits'
amp='0.035'
ra='266.417'
dec='-29.008'
t_exp='100000'

# Run the command
attgen_dither \
    Attitude=${att_file} \
    Amplitude=${amp} \
    SrcRA=${ra} \
    SrcDec=${dec} \
    Exposure=${t_exp}

There should now be an attitude file (`GC_AXIS_100000_att.fits`).

### 4.2. EVENTS FILE

Having prepared a complete `SIMPUT` file in Section 3 (`GC.simput`), and an attitude file in Section 4.1 (`GC_AXIS_100000_att.fits`), we can now create a simulated events file using the `SIXTE` tool `sixtesim`.

Edit and run the script `make_events.sh` in a `SIXTE`-initialized terminal:

In [ ]:
#!/bin/bash

# Input parameters
evt_file='GC_AXIS_100000_evt.fits'
xml_file='/Users/arash/astro/heasoft/sixte/share/sixte/instruments/axis/axis_baseline_all_chips.xml'
att_file='GC_AXIS_100000_att.fits'
roll='75.306'
sim_file='GC.simput'
t_exp='100000'

# Run the command
sixtesim \
    EvtFile=${evt_file} \
    XMLFile=${xml_file} \
    Attitude=${att_file} \
    rollangle=${roll} \
    Simput=${sim_file} \
    Exposure=${t_exp} \
    clobber=yes

There should now be an events file for each of the four AXIS detector chips: 

* `chip0_GC_AXIS_100000_evt.fits`

* `chip1_GC_AXIS_100000_evt.fits`

* `chip2_GC_AXIS_100000_evt.fits`

* `chip3_GC_AXIS_100000_evt.fits`

Use the `HEAsoft` tool `ftmerge` to combine them into a single file (`GC_AXIS_100000_evt.fits`):

In [ ]:
ftmerge \
    chip0_GC_AXIS_100000_evt.fits,chip1_GC_AXIS_100000_evt.fits,chip2_GC_AXIS_100000_evt.fits,chip3_GC_AXIS_100000_evt.fits \
    GC_AXIS_100000_evt.fits clobber=yes

There should now be a single events file that combines all four chips (`GC_AXIS_100000_evt.fits`). 

This is the required input to make an image file as described in Section 5.

## 5. GENERATE THE IMAGE FILE

The simulated events file from Section 5.2 (`GC_AXIS_100000_evt.fits`) can be made into an image file (`GC_AXIS_100000_img.fits`) using the SIXTE tool `imgev`.

Edit and run the script `make_image.sh` in a `SIXTE`-initialized terminal:

In [ ]:
#!/bin/bash

# Input parameters
evt_file='GC_AXIS_100000_evt.fits'
img_file='GC_AXIS_100000_img.fits'
coord='0'
projection='TAN'
naxis='2880'
cunit='deg'
ra='266.417'
dec='-29.008'
crpix='1440.5'
cdelt='2.4e-04'

# Run the command
imgev \
    EvtFile=${evt_file} \
    Image=${img_file} \
    CoordinateSystem=${coord} Projection=${projection} \
    NAXIS1=${naxis} NAXIS2=${naxis} \
    CUNIT1=${cunit} CUNIT2=${cunit} \
    CRVAL1=${ra} CRVAL2=${dec} \
    CRPIX1=${crpix} CRPIX2=${crpix} \
    CDELT1=-${cdelt} CDELT2=${cdelt} \
    clobber=yes

There should now be an image file (`GC_AXIS_100000_img.fits`) whose full window is shown below: 

<div>
<img src="img/GC_AXIS_100000_img.png" width="800"/>
</div>

The outline of the diffuse emission image from _Chandra_ is visible which suggests that its flux is slighltly higher than it should be.

Zooming in, and changing the axes to Galactic coordinates, shows the detail that can be achieved with AXIS in 100 ks of exposure time. This will be the typical depth allocated to this region of the GPS.

<div>
<img src="img/GC_AXIS_100000_img_zoom.png"/>
</div>

The figure below shows a zoomed view of the vicinity of Sgr A $^{\star}$ from _Chandra_ (left panel: 167 ks) compared with the simulated AXIS image (right panel: 100 ks). 

<div>
<img src="img/chandra_axis_gc_100ks.png"/>
</div>

## 6. GENERATE THE EXPOSURE MAP

The `SIXTE` tool `exposure_map` can create an exposure map using the XML and vignetting files, and other parameters.

Edit and run the script `make_exposure.sh` in a `SIXTE`-initialized terminal:

In [ ]:
#!/bin/bash

# Input parameters
ra='266.417'
dec='-29.008'
roll='75.306'
vig_file='/Users/arash/astro/heasoft/sixte/share/sixte/instruments/axis/sixte_calibration_files/axis_vignetting_20230821.fits'
xml_file='/Users/arash/astro/heasoft/sixte/share/sixte/instruments/axis/axis_baseline_all_chips.xml'
att_file='GC_AXIS_100000_att.fits'
exp_file='GC_AXIS_100000_exp.fits'
t_exp='100000'
t_start='0'
dt='1000'
coord='0'
projection='TAN'
naxis='2880'
cunit='deg'
crpix='1440.5'
cdelt='2.4e-04'

# Run the command
exposure_map \
    RA=${ra} Dec=${dec} rollangle=${roll} \
    Vignetting=${vig_file} \
    XMLFile=${xml_file} \
    Attitude=${att_file} \
    Exposuremap=${exp_file} \
    TSTART=${t_start} timespan=${t_exp} dt=${dt} \
    CoordinateSystem=${coord} projection_type=${projection} \
    NAXIS1=${naxis} NAXIS2=${naxis} \
    CUNIT1=${cunit} CUNIT2=${cunit} \
    CRVAL1=${ra} CRVAL2=${dec} \
    CRPIX1=${crpix} CRPIX2=${crpix} \
    CDELT1=-${cdelt} CDELT2=${cdelt} \
    clobber=yes


There should now be an exposure map (`GC_AXIS_100000_exp.fits`) as shown below:

<div>
<img src="img/GC_AXIS_100000_exp.png" width="500"/>
</div>

## 7. APPENDIX

### 7.1. GENERATE IMAGES OF DIFFERENT BANDS

It is assumed that all steps have been performed up to, and including, the creation of an events file in Section 4. 

We seek to create an RGB-like image for the AXIS simulation, similar to the publicly-available 3-color _Chandra_ image of Baganoff et al. (2009) as seen in Section 1.

Using the `HEAsoft` tool `fv`, open the RMF (`axis_ccd_20221101.rmf`) and verify the keV-to-PHA correspondence shown below:

* R-color image: [0.5; 3.3) keV; PHA [40; 320) 

* G-color image: [3.3; 4.7) keV; PHA [320; 460)

* B-color image: [4.7; 8] keV; PHA [460; 790)

The `HEAsoft` tool `ftcopy` will make a copy of the simulated events file (`GC_AXIS_100000_evt.fits`), while limiting the energy channels to the range of `pha_min` and `pha_max`.

Edit the following commands and paste them into a `HEAsoft`-initialized terminal:

In [ ]:
ftcopy \
    infile='GC_AXIS_100000_evt.fits[EVENTS][PHA >= 40 && PHA < 320]' \
    outfile='GC_AXIS_100000_evt_R.fits' \
    clobber=yes

ftcopy \
    infile='GC_AXIS_100000_evt.fits[EVENTS][PHA >= 320 && PHA < 460]' \
    outfile='GC_AXIS_100000_evt_G.fits' \
    clobber=yes

ftcopy \
    infile='GC_AXIS_100000_evt.fits[EVENTS][PHA >= 460 && PHA < 790]' \
    outfile='GC_AXIS_100000_evt_B.fits' \
    clobber=yes

There should now be three new events files from which to make a "color" image: 

- `GC_AXIS_100000_evt_R.fits`

- `GC_AXIS_100000_evt_G.fits`

- `GC_AXIS_100000_evt_B.fits`

Edit the following commands and paste them into a `SIXTE`-initialized terminal:

In [ ]:
export target='GC'
export inst='AXIS'
export t_exp='100000'
export ra='266.417'
export dec='-29.008'
export naxis='2880'
export crpix='1440.5'
export cdelt='2.4e-04'

imgev \
    EvtFile=${target}_${inst}_${t_exp}_evt_R.fits \
    Image=${target}_${inst}_${t_exp}_img_R.fits \
    CoordinateSystem=0 Projection=TAN \
    NAXIS1=${naxis} NAXIS2=${naxis} \
    CUNIT1=deg CUNIT2=deg \
    CRVAL1=${ra} CRVAL2=${dec} \
    CRPIX1=${crpix} CRPIX2=${crpix} \
    CDELT1=-${cdelt} CDELT2=${cdelt} \
    clobber=yes

imgev \
    EvtFile=${target}_${inst}_${t_exp}_evt_G.fits \
    Image=${target}_${inst}_${t_exp}_img_G.fits \
    CoordinateSystem=0 Projection=TAN \
    NAXIS1=${naxis} NAXIS2=${naxis} \
    CUNIT1=deg CUNIT2=deg \
    CRVAL1=${ra} CRVAL2=${dec} \
    CRPIX1=${crpix} CRPIX2=${crpix} \
    CDELT1=-${cdelt} CDELT2=${cdelt} \
    clobber=yes

imgev \
    EvtFile=${target}_${inst}_${t_exp}_evt_B.fits \
    Image=${target}_${inst}_${t_exp}_img_B.fits \
    CoordinateSystem=0 Projection=TAN \
    NAXIS1=${naxis} NAXIS2=${naxis} \
    CUNIT1=deg CUNIT2=deg \
    CRVAL1=${ra} CRVAL2=${dec} \
    CRPIX1=${crpix} CRPIX2=${crpix} \
    CDELT1=-${cdelt} CDELT2=${cdelt} \
    clobber=yes


This creates 3 new image files from which to make a "color" image: 

- `GC_AXIS_100000_img_R.fits`

- `GC_AXIS_100000_img_G.fits`

- `GC_AXIS_100000_img_B.fits`

These images can be loaded into `DS9`:

In [ ]:
ds9 -rgb -red GC_AXIS_100000_img_R.fits \
    -green GC_AXIS_100000_img_G.fits \
    -blue GC_AXIS_100000_img_B.fits &

<div>
<img src="img/GC_AXIS_100000_img_RGB.png"/>
</div>

### 7.2. FIND SIGNIFICANT DETECTIONS WITH WAVDETECT

It is assumed that all steps have been performed up to, and including, the creation of an image and an exposure map in Sections 5-6. 

In order to detect sources in the image with the `CIAO` tool `wavdetect`, we first need to create a PSF file that has the same dimensions as the image, with a PSF-radius of 1.25 $^{\prime\prime}$ that stays constant across the field.

Edit the following commands and paste them into a `CIAO`-initialized terminal:

In [ ]:
export prefix='GC_AXIS_100000'
export psf_size='1.25'

punlearn dmimgcalc

dmimgcalc ${prefix}_img.fits none ${prefix}_psf.fits op="imgout=((img1*0)+${psf_size})" clobber=yes
dmhedit ${prefix}_psf.fits none add BUNIT "arcsec"

There should now be a PSF file called `GC_AXIS_100000_psf.fits`.

We can now run `wavdetect` to help detect sources in the image (`GC_AXIS_100000_img.fits`).

Edit the following commands and paste them into a `CIAO`-initialized terminal:

In [ ]:
export prefix='GC_AXIS_100000'

punlearn wavdetect

wavdetect \
 infile=${prefix}_img.fits \
 outfile=${prefix}_wavsrc.fits \
 scellfile=${prefix}_scell.fits \
 imagefile=${prefix}_wavimg.fits \
 defnbkgfile=${prefix}_wavnbgd.fits \
 psffile=${prefix}_psf.fits \
 expfile=${prefix}_exp.fits \
 regfile=${prefix}_wavsrc.reg \
 scales="1.0 2.0 4.0 8.0 16.0" \
 clobber=yes

The files `GC_AXIS_100000_wavsrc.fits` and `GC_AXIS_100000_wavsrc.reg` contain the list of detected sources and their region file.

Use the `HEAsoft` tools `ftselect` to extract source detections above a given significance (e.g., 3), and `ftconvert` to dump the coordinates into a TXT file, which can be easily converted to a region file. 

In [ ]:
export prefix='GC_AXIS_100000'
export min_sig='3'

ftselect ${prefix}_wavsrc.fits temp.fits 'SRC_SIGNIFICANCE >= ${min_sig}' clobber=yes

ftconvert temp.fits ${prefix}_wavsrc_sig${min_sig}.txt RA,DEC - clobber=y

The image below shows the simulated AXIS image and the `wavdetect`-generated outlines of detected source regions for the inner GC:

<div>
<img src="img/GC_AXIS_100000_img_wavsrc.png"/>
</div>

The image below presents shows the simulated AXIS image as well as the list of Muno et al. (2009) _Chandra_ sources from 2 Ms of exposure time (magenta crosses), and sources detected with a significance above 3 $\sigma$ by AXIS after 100 ks of exposure time (yellow circles):

<div>
<img src="img/GC_AXIS_100000_img_zoom_3sig.png"/>
</div>

### 7.3. EXTRACT SOURCE SPECTRA

It is assumed that all steps have been performed up to, and including, the creation of an image and an exposure map in Sections 5-6. 

First, we create source and background extraction regions for a bright on-axis source:

* CXOU J174607.4-285951: (266.53122,-28.99759)

The `src_extract.reg` file has the following lines:

In [ ]:
# Region file format: DS9 version 4.1
global color=green width=1 
fk5
circle(266.53122,-28.99759,5.0")

The `bkg_extract.reg` file has the following lines:

In [ ]:
# Region file format: DS9 version 4.1
global color=green width=1 
fk5
annulus(266.53122,-28.99759,5.0",10.0")


We will use the `SIXTE` tool `makespec` to extract the source and background spectra from the events file.

Before doing so, we must add physical detector coordinates (_x_,_y_) to the events file.

This is done by the `SIXTE` tool `radec2xy`. 

Edit the following commands and paste them into a `SIXTE`-initialized terminal:

In [ ]:
export prefix='GC_AXIS_100000'
export ra='266.417'
export dec='-29.008'

radec2xy \
EvtFile=${prefix}_evt.fits \
projection=TAN \
RefRA=${ra} \
RefDec=${dec}

You can verify, with the HEAsoft tool `fv`, that the events file (`GC_AXIS_100000_evt.fits`) now includes columns for _x_ and _y_ detector pixel coordinates.

Edit the following commands and paste them into a `SIXTE`-initialized terminal; this will call the `SIXTE` tool `makespec` which extracts source and background spectra for the regions made earlier:

In [ ]:
export evt_file='GC_AXIS_100000_evt.fits'
export src_reg='src_extract.reg'
export src_spec='src_spec.pha'
export bkg_reg='bkg_extract.reg'
export bkg_spec='bkg_spec.pha'
export sixte_loc='/Users/arash/astro/heasoft/sixte/share/sixte/instruments/axis/sixte_calibration_files'
export rsp_file='${sixte_loc}/axis_ccd_20221101.rmf'
export arf_file='${sixte_loc}/axis_onaxis_20230701.arf'

makespec \
    EvtFile=${evt_file} \
    Spectrum=${src_spec} \
    EventFilter="regfilter(\"${src_reg}\")"  \
    RSPPath=${sixte_loc} \
    usepha=true \
    clobber=yes

makespec \
    EvtFile=${evt_file} \
    Spectrum=${bkg_spec} \
    EventFilter="regfilter(\"${bkg_reg}\")" \
    RSPPath=${sixte_loc} \
    usepha=true \
    clobber=yes

There should now be two spectral files; one for the source and one for the background:

* `src_spec.pha`

* `bkg_spec.pha`

The `HEAsoft` tool `grppha` will rebin these source and background spectra to a minimum of 20 cts/bin to permit $\chi^{2}$ statistics, and then group these spectra with the response files (`axis_ccd_20221101.rmf` and `axis_onaxis_20230701.arf`).

Note that the background spectrum needs to be scaled by a factor of 0.333 given its larger area compared with the source extraction region.

Edit the following commands and paste them into a `HEAsoft`-initialized terminal:

In [ ]:
export evt_file='GC_AXIS_100000_evt.fits'
export src_spec='src_spec.pha'
export bkg_spec='bkg_spec.pha'
export rsp_file='/Users/arash/astro/heasoft/sixte/share/sixte/instruments/axis/sixte_calibration_files/axis_ccd_20221101.rmf'
export arf_file='/Users/arash/astro/heasoft/sixte/share/sixte/instruments/axis/sixte_calibration_files/axis_onaxis_20230701.arf'
export min_cts='20'
export back_scale='0.333'

export grp_spec='src_spec_'${min_cts}'cts.pha'

grppha \
    infile=${src_spec} \
    outfile=${grp_spec} \
    comm="group min ${min_cts} & chkey BACKFILE ${bkg_spec} & chkey BACKSCAL ${back_scale} & chkey RESPFILE ${rsp_file} & chkey ANCRFILE ${arf_file} & exit" \
    clobber=Y


There should now be a grouped spectral file (`src_spec_20cts.pha`) which can be loaded for analysis in `Xspec`.

### 7.4. REPROCESS CHANDRA IMAGES

Download and reprocess the _Chandra_ observation to create an image of real data that can be compared with that of the AXIS simulation.

Edit the following commands and paste them into a `CIAO`-initialized terminal:

In [ ]:
download_chandra_obsid 3392
chandra_repro 3392 outdir=repro

fluximage repro/acisf03392_repro_evt2.fits repro/chandra_3392_img binsize=1 clobber=yes

In [ ]:
ds9 repro/acisf03392_repro_evt2.fits GC_AXIS_100000_img.fits &

The figure below shows a zoomed view of the vicinity of Sgr A $^{\star}$ from _Chandra_ (left panel: 167 ks) compared with the simulated AXIS image (right panel: 100 ks). 

<div>
<img src="img/chandra_axis_gc_100ks.png"/>
</div>